| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |

# Transforming Geochemical Data
##### *~10 min*

In [1]:
import pyrolite.geochem
import pandas as pd
import numpy as np
from pyrolite.util.synthetic import normal_frame

pd.set_option("precision", 3)  # smaller graphical outputs
 
df = normal_frame(columns=['CaO', 'MgO', 'SiO2', 'FeO','Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te']) * 100
df[['Ni', 'Ti', 'La', 'Lu', 'Te']] *= 10
df['Sr87/Sr86'] = 0.0700  / 0.0986 + np.random.randn(df.index.size) * 0.0001
df

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Sr87/Sr86
0,24.066,1.967,21.969,2.425,5.645,23.785,49.282,21.330,312.917,31.963,0.71
1,23.379,1.894,25.834,2.194,5.591,20.137,46.400,21.573,293.946,29.030,0.71
2,22.650,2.241,25.032,2.535,6.185,23.766,50.086,21.841,285.248,32.622,0.71
3,22.237,1.999,26.862,2.035,5.601,19.604,44.727,23.826,293.022,31.495,0.71
4,21.463,2.191,28.091,2.204,5.961,21.711,46.146,23.236,278.538,31.266,0.71
5,25.023,1.751,22.911,2.000,4.987,18.952,46.255,21.409,314.315,32.348,0.71
6,23.198,2.145,23.801,2.254,5.945,22.602,46.799,22.434,300.247,34.484,0.71
7,21.526,2.124,26.663,2.624,5.948,23.842,48.828,20.593,288.748,29.133,0.71
8,24.473,1.840,21.793,2.262,5.386,20.663,46.117,21.016,320.789,33.871,0.71
9,23.394,1.893,23.778,2.009,5.163,22.232,46.011,21.869,318.326,29.187,0.71


----
### Using Indexers, Scaling

You can also use these indexers for assignment, where the dimensionality of the dataset doesn't change. While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

In [2]:
from pyrolite.util.units import scale

els = df.pyrochem.elements.copy() # get a copy of just the elements from the dataframe, we'll then edit this version
els.pyrochem.elements *=  scale('ppm', 'wt%')

In [3]:
df.pyrochem.elements, els.pyrochem.elements

(       Ni      Ti      La       Lu      Te
 0  23.785  49.282  21.330  312.917  31.963
 1  20.137  46.400  21.573  293.946  29.030
 2  23.766  50.086  21.841  285.248  32.622
 3  19.604  44.727  23.826  293.022  31.495
 4  21.711  46.146  23.236  278.538  31.266
 5  18.952  46.255  21.409  314.315  32.348
 6  22.602  46.799  22.434  300.247  34.484
 7  23.842  48.828  20.593  288.748  29.133
 8  20.663  46.117  21.016  320.789  33.871
 9  22.232  46.011  21.869  318.326  29.187,
       Ni     Ti     La     Lu     Te
 0  0.002  0.005  0.002  0.031  0.003
 1  0.002  0.005  0.002  0.029  0.003
 2  0.002  0.005  0.002  0.029  0.003
 3  0.002  0.004  0.002  0.029  0.003
 4  0.002  0.005  0.002  0.028  0.003
 5  0.002  0.005  0.002  0.031  0.003
 6  0.002  0.005  0.002  0.030  0.003
 7  0.002  0.005  0.002  0.029  0.003
 8  0.002  0.005  0.002  0.032  0.003
 9  0.002  0.005  0.002  0.032  0.003)

----
### Molecular - Wt% Data

You can also use these indexers for assignment, where the dimensionality of the dataset doesn't change. While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

---
### Converting Chemical Components 

`pyrolite` provides some straightfoward methods to calcuate element-oxide conversions (e.g. to transform Ti abundance to TiO2 abudnance), assuming that the system is open to oxygen (i.e. in this case the extra oxygen will be added to the composition). This interface also allows the user to quickly add ratios and specify redox pairs at the same time. For example, we can transform a copy of our dataframe to include extra ratios and change some of our oxide components to elements:

In [4]:
df.pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "FeO", "Ca", "Te", "Na", "Na/Te", "MgO/SiO2"]
)

,Sr87/Sr86,MgO,SiO2,FeO,Ca,Te,Na,Na/Te,MgO/SiO2
0,0.71,1.967,21.969,2.425,17.200,31.963,4.188,0.131,0.090
1,0.71,1.894,25.834,2.194,16.709,29.030,4.148,0.143,0.073
2,0.71,2.241,25.032,2.535,16.188,32.622,4.589,0.141,0.090
3,0.71,1.999,26.862,2.035,15.892,31.495,4.155,0.132,0.074
4,0.71,2.191,28.091,2.204,15.340,31.266,4.422,0.141,0.078
5,0.71,1.751,22.911,2.000,17.884,32.348,3.700,0.114,0.076
6,0.71,2.145,23.801,2.254,16.579,34.484,4.410,0.128,0.090
7,0.71,2.124,26.663,2.624,15.385,29.133,4.413,0.151,0.080
8,0.71,1.840,21.793,2.262,17.491,33.871,3.995,0.118,0.084
9,0.71,1.893,23.778,2.009,16.720,29.187,3.830,0.131,0.080


In a similar way, we can also specify the molar speciation for redox species (so far just iron; others could be incorporated if they'll be useful). Here we adjust the total iron within our compositions (currently specified as FeO) to have a $Fe^{2+}/Fe^{3+}$ ratio of 9:1 (roughly what you might expect from a normal mantle-derived magma):

In [5]:
df.pyrochem.convert_chemistry(to=[{"FeO": 0.9, "Fe2O3": 0.1}])

,Sr87/Sr86,FeO,Fe2O3
0,0.71,2.183,0.270
1,0.71,1.975,0.244
2,0.71,2.282,0.282
3,0.71,1.831,0.226
4,0.71,1.984,0.245
5,0.71,1.800,0.222
6,0.71,2.029,0.251
7,0.71,2.362,0.292
8,0.71,2.036,0.251
9,0.71,1.808,0.223


----
<div class='alert alert-warning'> <b> <font size="+1">Checkpoint & Time Check</font></b><br>How are things going?</div>

----

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |